In [1]:
import fitz  # PyMuPDF
import os

def extract_text_from_pdfs(folder_path):
    all_texts = {}
    for file in os.listdir(folder_path):
        if file.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, file)
            doc = fitz.open(pdf_path)
            text = ""
            for page in doc:
                text += page.get_text()
            all_texts[file] = text
    return all_texts

pdf_folder = "/content/pdfs"  # PDF'leri buraya koy
pdf_texts = extract_text_from_pdfs(pdf_folder)

for name, text in pdf_texts.items():
    print(f"--- {name} ---")
    print(text[:500])  # ilk 500 karakter
    print("\n")


--- financial_report.pdf ---
Financial Report 2025 
  
1. Annual Revenue and Expenses 
The company achieved a total revenue of $12 million, reflecting a 15% growth 
compared to the previous year. Total operating expenses amounted to $8 million, 
including salaries, marketing, and administrative costs. 
  
2. Investment and Growth Strategy 
Investments were made in R&D and market expansion. A new product line contributed 
20% of the annual revenue. Future plans include entering new international markets 
and enhancing digita


--- company_policies.pdf ---
Company Policies 
  
1. Work Hours and Leave 
Employees are expected to work 9:00 AM to 6:00 PM, Monday through Friday. All leave 
requests must be submitted at least two weeks in advance and approved by the direct 
supervisor. Emergency leave will be handled on a case-by-case basis. Overtime work 
must be pre-approved and compensated according to the labor regulations. 
  
2. Data Security and Privacy 
All employees must adhere to com

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

import os

os.environ["OPENAI_API_KEY"] = ''

In [3]:
# Embedding oluşturucu
embeddings = OpenAIEmbeddings()

# Metinleri parçalara böl
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

docs = []
for name, text in pdf_texts.items():
    chunks = splitter.split_text(text)
    for i, chunk in enumerate(chunks):
        docs.append({"content": chunk, "source": name})

# Vektör veritabanı oluştur
texts = [d["content"] for d in docs]
metadatas = [{"source": d["source"]} for d in docs]

db = Chroma.from_texts(texts, embeddings, metadatas=metadatas)
retriever = db.as_retriever()

/tmp/ipython-input-2702217358.py:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [6]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.2)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever
)

In [7]:
queries = [
    "Şirket politikalarındaki veri gizliliği kurallarını özetle.",
    "Sözleşmede fesih koşulları nelerdir?",
    "Finansal raporda önerilen yatırım stratejilerini madde madde yaz.",
    "Tüm dokümanlardan genel bir özet çıkar."
]

for q in queries:
    print(f"❓Soru: {q}\n")
    print("💡Cevap:\n", qa_chain.run(q))
    print("\n" + "-"*80 + "\n")


❓Soru: Şirket politikalarındaki veri gizliliği kurallarını özetle.



/tmp/ipython-input-4209919534.py:10: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print("💡Cevap:\n", qa_chain.run(q))


💡Cevap:
 Şirket veri gizliliği politikaları, çalışanların gizli belgeleri güvenli bir şekilde işlemesini, şirket onaylı cihazların kullanılmasını ve şifre korumasını içerir. Hassas bilgilerin izinsiz paylaşımı kesinlikle yasaktır ve disiplin cezasına neden olabilir.

--------------------------------------------------------------------------------

❓Soru: Sözleşmede fesih koşulları nelerdir?

💡Cevap:
 Sözleşme, her iki tarafın da otuz günlük yazılı bir bildirimle feshedebileceği bir hüküm içermektedir. Ancak, şart ihlali durumunda fesih derhal gerçekleşebilir ve muhtemelen yasal yollara başvurulabilir.

--------------------------------------------------------------------------------

❓Soru: Finansal raporda önerilen yatırım stratejilerini madde madde yaz.

💡Cevap:
 Finansal raporda önerilen yatırım stratejileri şunlardır:

1. Teknolojiye yapılan yatırımların artırılması
2. Çalışanlar için eğitim programlarına yatırım yapılması
3. Yeni uluslararası pazarlara giriş yapılması
4. Dijital va

In [8]:
# 📝 7️⃣ Her PDF dosyasını tek tek özetle
summaries = {}
for name, text in pdf_texts.items():
    print(f"📄 {name} özetleniyor...")
    query = f"{name} adlı dokümanın en önemli kısımlarını kısa ve anlamlı şekilde özetle."
    summary = qa_chain.run(query)
    summaries[name] = summary
    print("✅ Özet tamamlandı!\n")

# 💾 8️⃣ Özetleri tek bir dosyada kaydet
summary_text = ""
for name, summ in summaries.items():
    summary_text += f"--- {name} ---\n{summ}\n\n"

output_path = "/content/summaries.txt"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(summary_text)

print(f"✅ Tüm özetler '{output_path}' dosyasına kaydedildi.")

📄 financial_report.pdf özetleniyor...
✅ Özet tamamlandı!

📄 company_policies.pdf özetleniyor...
✅ Özet tamamlandı!

📄 legal_contracts.pdf özetleniyor...
✅ Özet tamamlandı!

✅ Tüm özetler '/content/summaries.txt' dosyasına kaydedildi.
